In [7]:
api_key = 'RGAPI-314e77e7-1963-4196-96b5-cf8c8d0777b9'

In [15]:
import requests
import os
from psycopg2 import pool
from dotenv import load_dotenv
from psycopg2.extras import Json, DictCursor
import time

In [9]:
r = requests.get("https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/clu/clu1?api_key=" + api_key)
print(r.json())

{'puuid': 'R8_dzuvxZm0VsqU-paXM3wijLavgPmiZDmKIHFm8HddnUGvsgHTlCQuWdlHNRJlBIaEFyZYD87Avug', 'gameName': 'clu', 'tagLine': 'clu1'}


In [10]:
# setsuko, soju, dishsoap
streamers = ['e62ZthNQBc1bjVfXrjDvNIQV-0Bd7GUeaRv_fiZqx5nc5OXaY6OAmM4wav043AJgxKQShtA1s9IIOg', 'hVi-4fc19ymaRf4HvUmHNkZCoALJPUXgGKcKGgBlLZTqYZRVtQwglGQCIBj0jou0wA35CcH8sdeEgw', 'NUnja2Bl0CPRiimqr1PesnOnDsxXa6eAg5xRhNVBdxgAzBopnpcWNTZZ9r9YjNhEtf5tx2DkvXdJ9Q']

In [11]:
# for streamer_puuid in streamers:
#   matches = requests.get("https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/" + streamer_puuid + "&api_key=" + api_key)


In [12]:


# Load .env file
load_dotenv()

# Get the connection string from the environment variable
connection_string = os.getenv('DATABASE_URL')

# Create a connection pool
connection_pool = pool.SimpleConnectionPool(
    1,  # Minimum number of connections in the pool
    10,  # Maximum number of connections in the pool
    connection_string
)

# Check if the pool was created successfully
if connection_pool:
    print("Connection pool created successfully")

# Get a connection from the pool
conn = connection_pool.getconn()

# Create a cursor object
cur = conn.cursor()

# Execute SQL commands to retrieve the current time and version from PostgreSQL
cur.execute('SELECT NOW();')
time = cur.fetchone()[0]

cur.execute('SELECT version();')
version = cur.fetchone()[0]

# Close the cursor and return the connection to the pool
cur.close()
connection_pool.putconn(conn)

# Close all connections in the pool
connection_pool.closeall()

# Print the results
print('Current time:', time)
print('PostgreSQL version:', version)

Connection pool created successfully
Current time: 2024-11-12 00:23:18.013938+00:00
PostgreSQL version: PostgreSQL 16.4 on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit


In [16]:
for current_puuid in streamers:
  # Connect to the DB
  # Load .env file
  load_dotenv()

  # Get the connection string from the environment variable
  connection_string = os.getenv('DATABASE_URL')

  # Create a connection pool
  connection_pool = pool.SimpleConnectionPool(
      1,  # Minimum number of connections in the pool
      10,  # Maximum number of connections in the pool
      connection_string
  )

  # Check if the pool was created successfully
  if connection_pool:
      print("Connection pool created successfully")

  # Get a connection from the pool
  conn = connection_pool.getconn()
  conn.autocommit = True

  # Create a cursor object
  cur = conn.cursor(cursor_factory=DictCursor)


  # get recent match data
  matches = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/by-puuid/' + current_puuid + '/ids?start=0&count=20&api_key=' + api_key).json()

  # add each match to the db
  for match in matches:
    # first check not in the db
    
    cur.execute('SELECT 1 FROM matches WHERE match_id=%s', (match, ))
    if cur.fetchall():
      continue
    


    match_dict = {}
    match_data = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key)

    if match_data.ok:
      match_data = match_data.json()
    else:
      print("sleeping 120s")
      time.sleep(120)
      match_data = requests.get('https://americas.api.riotgames.com/tft/match/v1/matches/' + match + '?api_key=' + api_key).json()


    #get metadata
    patch = match_data['info']['game_version'][-6:-1]
    game_datetime = match_data['info']['game_datetime']

    game_information = match_data['info']['participants']
    players = []
    # print(game_information[0])
    for board in game_information:
      curr_dict = {}
      curr_dict['augments'] = board['augments']
      curr_dict['level'] = board['level']
      curr_dict['placement'] = board['placement']
      curr_dict['traits'] = board['traits']
      curr_dict['units'] = board['units']
      curr_dict['puuid'] = board['puuid']
      curr_dict['gold_left'] = board['gold_left']

      r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' + curr_dict['puuid'] + '?api_key=' + api_key)
      if r.ok:
        r = r.json()
      else:
        print("sleeping 120s")
        time.sleep(120)
        r = requests.get('https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/' + curr_dict['puuid'] + '?api_key=' + api_key).json()
      curr_dict['username_tagline']= r['gameName'] + ' #' + r["tagLine"]

      match_dict[r['gameName'] + ' #' + r["tagLine"]] = curr_dict.copy()
      players.append(r['gameName'] + ' #' + r["tagLine"])

    # add it into the db
  
    cur.execute('INSERT into matches (match_id, patch, game_datetime, match_data, players) values (%s, %s, %s, %s, %s)', (match, patch, game_datetime, Json(match_dict), players))
    print('inserted', match)



  # Commit changes to the db

  cur.close()
  connection_pool.putconn(conn)

  # Close all connections in the pool
  connection_pool.closeall()


Connection pool created successfully
inserted NA1_5131711742
inserted NA1_5131694229
inserted NA1_5131672537
inserted NA1_5131655036
inserted NA1_5131632927
inserted NA1_5131606528
inserted NA1_5131581142
inserted NA1_5131554751
inserted NA1_5131519989
Connection pool created successfully
inserted NA1_5143191535
sleeping 120s
inserted NA1_5143175262
inserted NA1_5143164964
inserted NA1_5143152150
inserted NA1_5143142131
inserted NA1_5143127402
inserted NA1_5143114553
inserted NA1_5139246343
inserted NA1_5135873144
inserted NA1_5135546046
inserted NA1_5135509155
inserted NA1_5135470479
inserted NA1_5134935794
sleeping 120s


KeyboardInterrupt: 

In [21]:
def getStreamerData():
    load_dotenv()

    # Get the connection string from the environment variable
    connection_string = os.getenv('DATABASE_URL')

    # Create a connection pool
    connection_pool = pool.SimpleConnectionPool(
        1,  # Minimum number of connections in the pool
        10,  # Maximum number of connections in the pool
        connection_string
    )

    # Check if the pool was created successfully
    if connection_pool:
        print("Connection pool created successfully")

    # Get a connection from the pool
    conn = connection_pool.getconn()
    conn.autocommit = True

    # Create a cursor object
    cur = conn.cursor(cursor_factory=DictCursor)

    cur.execute("SELECT * FROM matches WHERE 'LemonVip #NA1' = ANY(players) ORDER BY game_datetime DESC")
    print(cur.fetchall())

    cur.close()
    connection_pool.putconn(conn)

    # Close all connections in the pool
    connection_pool.closeall()

getStreamerData()

Connection pool created successfully
[['NA1_5149100514', '14.22', '1731018571470', {'Knav #NA1': {'level': 8, 'puuid': 'HPJhFXY_2UL7hDHNExlkBdLo-PvPbuQidP5_SIS3hr4WNQhBrp7JQTmtpmuGtDXYa1tXLx-0TGjiWQ', 'units': [{'name': '', 'tier': 3, 'rarity': 0, 'itemNames': ['TFT_Item_GiantsBelt'], 'character_id': 'TFT12_Elise'}, {'name': '', 'tier': 1, 'rarity': 0, 'itemNames': [], 'character_id': 'TFT12_Ziggs'}, {'name': '', 'tier': 3, 'rarity': 1, 'itemNames': ['TFT_Item_GuinsoosRageblade', 'TFT_Item_GuinsoosRageblade', 'TFT_Item_MadredsBloodrazor'], 'character_id': 'TFT12_Cassiopeia'}, {'name': '', 'tier': 3, 'rarity': 1, 'itemNames': ['TFT_Item_MadredsBloodrazor', 'TFT_Item_SpearOfShojin', 'TFT_Item_AdaptiveHelm'], 'character_id': 'TFT12_Syndra'}, {'name': '', 'tier': 3, 'rarity': 1, 'itemNames': [], 'character_id': 'TFT12_Shyvana'}, {'name': '', 'tier': 3, 'rarity': 2, 'itemNames': ['TFT_Item_DragonsClaw', 'TFT_Item_NightHarvester', 'TFT_Item_GargoyleStoneplate'], 'character_id': 'TFT12_Swain'